In [1]:
# !pip install numpy pandas tqdm tensorflow torch transformers qdrant-client

In [2]:
import numpy as np
import pandas as pd

import torch
from transformers import AutoTokenizer, AutoModel

from tqdm import tqdm

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

DIR = '/kaggle/input/aaa-project-search/'

cpu


In [3]:
df = pd.read_hdf(DIR + 'search_relevance_dataset_v1.hdf', 'table')
df.drop(columns=['query_id', 'query_category_id', 'query_microcat_id', 'query_location_id', 'item_id'], inplace=True)
df.head(3)

,query_text,title,description,keywords,target
0,2108 ссср,Советские бутыли канистры 60-80-х СССР ваз газ...,Для ценителей и понимающих.\n\nПодробные фотог...,стопсфинксstop,0
1,2108 ссср,Ваз 2108 СССР цвет салатовый 1/43 идеальный,красивый салатовый цвет\nвсе детали в наличии\...,модель,1
2,2108 ссср,Модели советских машин ваз 2102 почта М 1/43 №10,Продается модель автомобиля ВАЗ 2102 почта . ...,модель,1


In [4]:
tokenizer = AutoTokenizer.from_pretrained('cointegrated/rubert-tiny2')
model = AutoModel.from_pretrained('cointegrated/rubert-tiny2').to(device)

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy() # (312, )

q, p, n = df.iloc[0, 1], df.iloc[0, 0], df.iloc[42, 0]

((embed_bert_cls(q, model, tokenizer) - embed_bert_cls(p, model, tokenizer))**2).sum(),\
((embed_bert_cls(q, model, tokenizer) - embed_bert_cls(n, model, tokenizer))**2).sum()

(1.0305362, 1.1124296)

In [6]:
from qdrant_client import QdrantClient

client = QdrantClient(":memory:")

In [7]:
from qdrant_client.http.models import Distance, VectorParams

client.recreate_collection(
    collection_name='test_collection', 
    vectors_config=VectorParams(size=312, distance=Distance.DOT),
)

True

In [8]:
from qdrant_client.http.models import CollectionStatus

collection_info = client.get_collection(collection_name='test_collection')

assert collection_info.status == CollectionStatus.GREEN
assert collection_info.vectors_count == 0

In [10]:
from qdrant_client.http.models import PointStruct

points = []
for row in tqdm(df.itertuples(), total=len(df)):
    points.append(
        PointStruct(
            id=row.Index,
            vector=list(map(float, embed_bert_cls(row.title, model, tokenizer))), # list(map(float.. to make proper type
            payload={
                'title': row.title,
                'description': row.description,
                'keywords': row.keywords,
            },
        )
    )

100%|██████████| 161429/161429 [15:02<00:00, 178.94it/s]


In [11]:
operation_info = client.upsert(
    collection_name='test_collection',
    wait=True,
    points=points
)

In [12]:
search_result = client.search(
    collection_name='test_collection',
    query_vector=list(map(float, embed_bert_cls(q, model, tokenizer))), 
    limit=10
)

In [13]:
print(q)
search_result

Советские бутыли канистры 60-80-х СССР ваз газ заз


[ScoredPoint(id=0, version=0, score=0.9999999749774325, payload={'title': 'Советские бутыли канистры 60-80-х СССР ваз газ заз', 'description': 'Для ценителей и понимающих.\n\nПодробные фотографии пришлю по запросу.\nВозможна отправка по РФ ( Почта/СДЭК )\n\nДля декора , антуража , ретро выставок , экспозиций или съемок фильмов.\n\nКак аксессуар для вашего старинного авто или гаража .\n\nПластиковые бутыли в которых продавались и хранились различные пищевые и непищевые жидкости ( тосол , охлаждающая жидкость , моторное масло М-8 , ТАД-17 , тормозная жидкость ) в 60-80-х годах.\n\nАвтоваз FIAT 124 Жигули ВАЗ 2101 копейка 21011 21013 Лада 2102 2103 LADA 2104 2105 Аксессуар 2106 21063 Ретро 2107 2108 Нива 2109 2121\nЗАПОРОЖЕЦ 965 горбатый 966 968 А 968 М Луаз РАФ\nЗМА 400 401 МЗМА 402 403 Москвич 407 408 АЗЛК 412 2138 Сувенир 2140 2125 Фляга 2715 2141 Бутылка  426 427 Ретро 2137\nВолга ПОБЕДА М-20 21 VOLGA 22 24 ЗИМ 2402 24-02 ЗИС 2410 24-10 ЗИЛ 69 12 13 14 Чайка 31029 3102 ГАЗ-21 ГАЗ-24 3